## Authenticate to Google Earth Engine

In [1]:
import ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='ee-raman')

In [2]:
def get_feature_collection(asset_id):
    """Check if an asset exists, and load it as a FeatureCollection if it does.
    Otherwise, return an empty FeatureCollection.
    
    Args:
        asset_id (str): The Earth Engine asset ID.
        
    Returns:
        ee.FeatureCollection: The loaded FeatureCollection or an empty one.
    """
    try:
        # Get asset information to check existence
        ee.data.getAsset(asset_id)
        print(f"Asset '{asset_id}' exists. Loading FeatureCollection.")
        return ee.FeatureCollection(asset_id)
    except Exception as e:
        print(f"Asset '{asset_id}' does not exist. Returning empty FeatureCollection.")
        return ee.FeatureCollection([])

In [3]:
directory = "Area_bheramgarh"
directory = "Area_jamkhed"
directory = "Area_Peddapally"

mapping = {
    "farm": 1,
    "plantation": 2,
    "scrubland": 3,
    "rest": 0
}
reversed_mapping = {v: k for k, v in mapping.items()}
reversed_ee_mapping = ee.Dictionary(reversed_mapping)

easy_farm = [
    ee.Filter.gte("rect", 0.67),
    ee.Filter.gt("size", 500),
    ee.Filter.lt("size", 2000),
    ee.Filter.lt("ent", 1)
    ]
easy_scurbland = [
    ee.Filter.gte("size", 60000),
    ee.Filter.gt("red", 0.9)
    ]
easy_plantation = [
    ee.Filter.lt("area", 20000),
    ee.Filter.gt("area", 1000)
]


all = get_feature_collection("projects/ee-raman/assets/" + directory + "_boundaries")
farm = all.filter(ee.Filter.And(*easy_farm))
scrubland = all.filter(ee.Filter.And(easy_scurbland))
plantation = all.filter(ee.Filter.eq("class", "plantation")).map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.And(easy_plantation))

label_image = ee.Image(0).rename("label")
farm_mask = label_image.clip(farm).mask()
scrubland_mask = label_image.clip(scrubland).mask()
plantation_mask = label_image.clip(plantation).mask()

label_image = label_image.where(farm_mask, mapping["farm"]).where(scrubland_mask, mapping["scrubland"]).where(plantation_mask, mapping["plantation"])
ts_data = ee.Image("projects/ee-raman/assets/ts_data_" + directory)

# Classes to sample (exclude background = 0)
class_values = [(1,20000), (2,20000), (3,20000)]

# Empty list to store samples
samples_list = []

for class_val, points in class_values:
    # Create a mask for the class
    class_mask = label_image.eq(class_val)
    
    # Mask the ts_image to only include pixels of this class
    masked_ts = ts_data.updateMask(class_mask)
    
    # Sample uniformly from the masked image
    class_samples = masked_ts.addBands(label_image.rename('class')) \
        .stratifiedSample(
            numPoints=points,  # adjust as needed
            classBand='class',
            classValues=[class_val],
            classPoints=[points],  # adjust per class
            scale=10,
            region=ts_data.geometry(),
            seed=42,
            geometries=True
        )
    
    samples_list.append(class_samples)

all_samples = samples_list[0].merge(samples_list[1]).merge(samples_list[2])

Asset 'projects/ee-raman/assets/Area_Peddapally_boundaries' exists. Loading FeatureCollection.


In [4]:
classifier = ee.Classifier.smileRandomForest(50).train(
    features=all_samples,
    classProperty='class',
    inputProperties=ts_data.bandNames()
)
classified = ts_data.classify(classifier)

In [5]:
def assign_mode_label(feature):
    class_values = classified.reduceRegion(
        reducer=ee.Reducer.mode(),
        geometry=feature.geometry(),
        scale=30,  # Adjust scale as per resolution
        bestEffort=True
    )
    return feature.set('class', class_values.get('classification'))

# Apply function to test features
all_labels = all.map(assign_mode_label).filter(ee.Filter.notNull(['class']))
all_labels = all_labels.map(lambda x: x.set('class', reversed_ee_mapping.get(ee.Number(x.get("class")).int())))

In [6]:
task = ee.batch.Export.table.toAsset(
    collection=all_labels,
    description='Classification',
    assetId="projects/ee-raman/assets/" + directory + "_boundaries_refined"
)

# Start the task
task.start()

In [15]:
import geemap

#final_lulc_img = ee.Image("projects/ee-raman/assets/" + filename_prefix)
#final_lulc_img_corrected = ee.Image("projects/ee-raman/assets/" + filename_prefix + "_corrected_slide")
#all = get_feature_collection("projects/ee-raman/assets/all_" + suffix)
#farm = all.filter(ee.Filter.eq("class", "farm"))
#scrubland = all.filter(ee.Filter.eq("class", "scrubland"))
#plantation = all.filter(ee.Filter.eq("class", "plantation"))

m = geemap.Map()
m.centerObject(classified)


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 1,
    'max': 3,
    'palette': palette_
}

#m.addLayer(final_lulc_img.select("predicted_label"), vis_params, 'LULC')
#m.addLayer(final_lulc_img_corrected.select("predicted_label"), vis_params_corrected, 'LULC_corrected')
m.addLayer(classified, vis_params_, 'predictions')

m.add_legend(keys=labels, colors=palette_corrected, position="bottomleft")
#m.addLayer(classified.select("classification"), vis_params_, "rest_with_labels" )
m.addLayer(farm, {}, 'farm')
m.addLayer(scrubland, {}, 'scrubland')
m.addLayer(all, {}, 'all')
m.addLayer(plantation.map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.lt("area", 20000)).filter(ee.Filter.gt("area", 1000)), {}, 'plantation')
#m.addLayer(rest, {}, 'rest')
#m.addLayer(rest_all.filter(ee.Filter.eq("predicted_class", 0)), {}, 'predicted_farm')
#m.addLayer(rest_all.filter(ee.Filter.eq("predicted_class", 1)), {}, 'predicted_scrubland')
#m.addLayer(rest_all.filter(ee.Filter.eq("predicted_class", 2)), {}, 'predicted_plantation')
#m.addLayer(aatif_plantation, {}, 'aatif_plantation')


m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
directory = "Area_bheramgarh"
directory = "Area_jamkhed"
directory = "Area_peddapally"

suffix = directory.split("_")[-1]
mapping = {
    "farm": 1,
    "plantation": 2,
    "scrubland": 3,
    "rest": 0
}
reversed_mapping = {v: k for k, v in mapping.items()}
reversed_ee_mapping = ee.Dictionary(reversed_mapping)

def get_class_and_mapping(vectors, vector_name):
    vector_of_class = vectors.filter(ee.Filter.eq('class', vector_name))
    vector_of_class_with_map = vector_of_class.map(lambda f: f.set('class', mapping[vector_name]))
    return vector_of_class, vector_of_class_with_map

all = get_feature_collection("projects/ee-raman/assets/" + directory + "_boundaries")
farm, farm_with_map = get_class_and_mapping(all, "farm")
scrubland, scrubland_with_map = get_class_and_mapping(all, "scrubland")
plantation, plantation_with_map = get_class_and_mapping(all, "plantation")
rest, rest_with_map = get_class_and_mapping(all, "rest")

plantation = plantation.map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.lt("area", 20000)).filter(ee.Filter.gt("area", 1000))

label_image = ee.Image(0).rename("label")
farm_mask = label_image.clip(farm).mask()
scrubland_mask = label_image.clip(scrubland).mask()
plantation_mask = label_image.clip(plantation).mask()

label_image = label_image.where(farm_mask, mapping["farm"]).where(scrubland_mask, mapping["scrubland"]).where(plantation_mask, mapping["plantation"])
ts_data = ee.Image("projects/ee-raman/assets/ts_data_" + directory)

# Classes to sample (exclude background = 0)
class_values = [(1,20000), (2,20000), (3,20000)]

# Empty list to store samples
samples_list = []

for class_val, points in class_values:
    # Create a mask for the class
    class_mask = label_image.eq(class_val)
    
    # Mask the ts_image to only include pixels of this class
    masked_ts = ts_data.updateMask(class_mask)
    
    # Sample uniformly from the masked image
    class_samples = masked_ts.addBands(label_image.rename('class')) \
        .stratifiedSample(
            numPoints=points,  # adjust as needed
            classBand='class',
            classValues=[class_val],
            classPoints=[points],  # adjust per class
            scale=10,
            region=ts_data.geometry(),
            seed=42,
            geometries=True
        )
    
    samples_list.append(class_samples)

all_samples = samples_list[0].merge(samples_list[1]).merge(samples_list[2])

#training_features = farm_with_map.merge(scrubland_with_map).merge(plantation_with_map)
